# Chapter 5: Text Classification

In [22]:
#!pip install torchtext==0.9.1
#!pip install torch==1.8.1
torchtext.__version__

ERROR: Could not find a version that satisfies the requirement torchtext==0.9.1 (from versions: 0.1.1, 0.2.0, 0.2.1, 0.2.3, 0.3.1, 0.4.0, 0.5.0, 0.6.0, 0.10.0, 0.11.2, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.0, 0.15.1)
ERROR: No matching distribution found for torchtext==0.9.1
ERROR: Could not find a version that satisfies the requirement torch==1.8.1 (from versions: 1.9.0, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0)
ERROR: No matching distribution found for torch==1.8.1


'0.15.1'

In [23]:
import spacy
import torchtext
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy import data

ModuleNotFoundError: No module named 'torchtext.legacy'

## Loading & Data Cleaning

In [17]:
device = "mps"

In [18]:
# You'll probably need to use the 'python' engine to load the CSV
# tweetsDF = pd.read_csv("training.1600000.processed.noemoticon.csv", header=None)
tweetsDF = pd.read_csv("../tweets_data/training.1600000.processed.noemoticon.csv",
                       engine="python", header=None, encoding="ISO-8859-1")

KeyboardInterrupt: 

In [7]:
tweetsDF[0].value_counts()

0    800000
4    800000
Name: 0, dtype: int64

In [8]:
tweetsDF["sentiment_cat"] = tweetsDF[0].astype('category')
tweetsDF["sentiment"] = tweetsDF["sentiment_cat"].cat.codes
tweetsDF.to_csv("../tweets_data/train-processed.csv", header=None, index=None)      
tweetsDF.sample(10000).to_csv("../tweets_data/train-processed-sample.csv", header=None, index=None) 


In [21]:
LABEL = data.LabelField()
TWEET = data.Field('spacy', tokenizer_language='en_core_web_sm', lower=True)

fields = [('score',None), ('id',None), ('date',None), ('query',None),
          ('name',None), ('tweet', TWEET), ('category',None), ('label',LABEL)]

AttributeError: module 'torchtext.data' has no attribute 'Field'

## Create our Dataset and DataLoaders

In [24]:
twitterDataset = data.dataset.TabularDataset(
        path="train-processed-sample.csv", 
        format="CSV", 
        fields=fields,
        skip_header=False)

AttributeError: module 'torchtext.data' has no attribute 'dataset'

In [40]:
(train, test, valid) = twitterDataset.split(split_ratio=[0.6,0.2,0.2],
                                            stratified=True, strata_field='label')

(len(train),len(test),len(valid))

(6000, 2000, 2000)

In [41]:
vocab_size = 20000
TWEET.build_vocab(train, max_size = vocab_size)
LABEL.build_vocab(train)
TWEET.vocab.freqs.most_common(10)

[('i', 3742),
 ('!', 3315),
 ('.', 3084),
 (' ', 2175),
 ('to', 2115),
 ('the', 2022),
 (',', 1823),
 ('a', 1461),
 ('my', 1205),
 ('it', 1197)]

In [42]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test),
    batch_size = 32,
    device = device,
    sort_key = lambda x: len(x.tweet),
    sort_within_batch = False)

## Our First LSTM

In [43]:
class OurFirstLSTM(nn.Module):
    def __init__(self, hidden_size, embedding_dim, vocab_size):
        super(OurFirstLSTM, self).__init__()
    
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(input_size=embedding_dim,  
                hidden_size=hidden_size, num_layers=1)
        self.predictor = nn.Linear(hidden_size, 2)

    def forward(self, seq):
        output, (hidden,_) = self.encoder(self.embedding(seq))
        preds = self.predictor(hidden.squeeze(0))
        return preds

model = OurFirstLSTM(100,300, 20002)
model.to(device)

OurFirstLSTM(
  (embedding): Embedding(20002, 300)
  (encoder): LSTM(300, 100)
  (predictor): Linear(in_features=100, out_features=2, bias=True)
)

## Training

In [44]:
optimizer = optim.Adam(model.parameters(), lr=2e-2)
criterion = nn.CrossEntropyLoss()

def train(epochs, model, optimizer, criterion, train_iterator, valid_iterator):
    for epoch in range(1, epochs+1):
     
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch_idx, batch in enumerate(train_iterator):
            optimizer.zero_grad()
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * batch.tweet.size(0)
        training_loss /= len(train_iterator)
 
        
        model.eval()
        for batch_idx,batch in enumerate(valid_iterator):
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            valid_loss += loss.data.item() * batch.tweet.size(0)
 
        valid_loss /= len(valid_iterator)
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}'.format(epoch, training_loss, valid_loss))

In [45]:
train(5, model, optimizer, criterion, train_iterator, valid_iterator)        

Epoch: 1, Training Loss: 24.47, Validation Loss: 14.04
Epoch: 2, Training Loss: 23.81, Validation Loss: 14.57
Epoch: 3, Training Loss: 23.25, Validation Loss: 15.69
Epoch: 4, Training Loss: 23.12, Validation Loss: 16.16
Epoch: 5, Training Loss: 21.71, Validation Loss: 18.80


## Making predictions

In [46]:
def classify_tweet(tweet):
    categories = {0: "Negative", 1:"Positive"}
    processed = TWEET.process([TWEET.preprocess(tweet)])
    processed = processed.to(device)
    model.eval()
    return categories[model(processed).argmax().item()]

## Data Augmentation

In [ ]:
def random_deletion(words, p=0.5):
    if len(words) == 1:
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    if len(remaining) == 0:
        return [random.choice(words)]
    else:
        return remaining

In [ ]:
def random_swap(sentence, n=5):
    length = range(len(sentence))
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1]
    return sentence

In [ ]:
# Note: you'll have to define remove_stopwords() and get_synonyms() elsewhere

def random_insertion(sentence,n):
    words = remove_stopwords(sentence)
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym)
    return sentence

In [ ]:
# Install googletrans version 3.1.0a0 (temporary fix for #57)
!pip install googletrans==3.1.0a0

In [ ]:
import googletrans
import random

translator = googletrans.Translator()

sentences = ['The cat sat on the mat']

translations_fr = translator.translate(sentences, dest='fr')
fr_text = [t.text for t in translations_fr] 
translations_en = translator.translate(fr_text, dest='en')
en_text = [t.text for t in translations_en]
print(en_text)   

available_langs = list(googletrans.LANGUAGES.keys())
tr_lang = random.choice(available_langs)
print(f"Translating to {googletrans.LANGUAGES[tr_lang]}")

translations = translator.translate(sentences, dest=tr_lang)
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=tr_lang, dest='en')
en_text = [t.text for t in translations_en_random]
print(en_text)